<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 03 - Creating and Querying a Data Mart

In this tutorial you will learn how to create and query a Data Mart using Virtualized Data in a SQL Server big data cluster. 

**TODO:** Complete Tutorial 

In [10]:
/* Show the External Table we want to keep */
SELECT TOP 10 * 
FROM WideWorldImporters.dbo.web_clickstreams_hdfs;
GO

(10 rows affected)

Total execution time: 00:00:02.1734889

wcs_click_date_sk,wcs_click_time_sk,wcs_sales_sk,wcs_item_sk,wcs_web_page_sk,wcs_user_sk
36890,41285,NULL,6245,34,NULL
36890,23115,NULL,13852,34,NULL
36890,17702,NULL,15975,34,NULL
36890,62676,NULL,2119,34,NULL
36890,34267,NULL,10273,34,NULL
36890,8502,NULL,17790,34,NULL
36890,54340,NULL,3453,34,NULL
36890,54370,NULL,6372,34,NULL
36890,6578,NULL,17203,34,NULL
36890,75088,NULL,4891,34,NULL


In [1]:
IF NOT EXISTS(SELECT * FROM sys.external_data_sources WHERE name = 'SqlDataPool')
  CREATE EXTERNAL DATA SOURCE SqlDataPool
  WITH (LOCATION = 'sqldatapool://service-mssql-controller:8080/datapools/default');

Commands completed successfully.

Total execution time: 00:00:00.0455778

In [20]:
CREATE EXTERNAL TABLE [web_clickstream_clicks_data_pool]
("wcs_click_date_sk" BIGINT , "wcs_click_time_sk" BIGINT , "wcs_sales_sk" BIGINT , "wcs_item_sk" BIGINT, "wcs_web_page_sk" BIGINT , "wcs_user_sk" BIGINT)
WITH
(
    DATA_SOURCE = SqlDataPool,
    DISTRIBUTION = ROUND_ROBIN
);

: Msg 15151, Level 16, State 1, Line 1
Cannot drop the EXTERNAL TABLE 'web_clickstream_clicks_data_pool', because it does not exist or you do not have permission.

Total execution time: 00:00:00.0347624

Commands completed successfully.

Total execution time: 00:00:00.1059128

In [21]:
DECLARE @db_name SYSNAME = 'WideWorldImporters'
DECLARE @schema_name SYSNAME = 'dbo'
DECLARE @table_name SYSNAME = 'web_clickstream_clicks_data_pool'
DECLARE @query NVARCHAR(MAX) = 'SELECT * FROM WideWorldImporters.dbo.web_clickstreams_hdfs;'

EXEC model..sp_data_pool_table_insert_data @db_name, @schema_name, @table_name, @query


In [22]:
SELECT count(*) FROM [dbo].[web_clickstream_clicks_data_pool]
SELECT TOP 10 * FROM [dbo].[web_clickstream_clicks_data_pool]

(1 row affected)

(10 rows affected)

Total execution time: 00:00:03.4759142

(No column name)
512000


wcs_click_date_sk,wcs_click_time_sk,wcs_sales_sk,wcs_item_sk,wcs_web_page_sk,wcs_user_sk
37019,85961,NULL,3971,14,NULL
37019,24228,NULL,3716,14,NULL
37019,62814,NULL,3143,14,NULL
37019,60131,NULL,13202,14,NULL
37019,48860,NULL,13714,14,NULL
37019,70453,NULL,248,14,NULL
37019,72847,NULL,6721,14,NULL
37019,50429,NULL,17647,14,NULL
37019,60478,NULL,16914,14,NULL
37019,63148,NULL,6664,14,NULL
